# Astro Lab Book 2025

In [2]:
import astropy
import ccdproc
from astropy.nddata import CCDData
from astropy.time.time_helper.function_helpers import linspace
from ccdproc import ccd_process
import numpy as np
from pathlib import Path
import ccdproc as ccdp
from astropy import units as u
import matplotlib.pyplot as plt
import photutils.background as bg
from astropy.convolution import convolve
from photutils.segmentation import make_2dgaussian_kernel
from photutils.segmentation import detect_sources
from photutils.segmentation import deblend_sources
from photutils.utils import calc_total_error
from photutils.segmentation import SourceCatalog
from photutils.aperture import CircularAperture, aperture_photometry
from photutils.background import MedianBackground
import os
import glob

### Combined Bias

In [2]:
raw_data = Path('data/bias')

calibrated_biases = glob.glob(str(raw_data / '*.fit*'))

combined_bias = ccdp.combine(calibrated_biases,
                             method='average',
                             mem_limit=350e6,
                             unit='adu'
                            )

combined_bias.meta['combined'] = True
combined_bias.write(raw_data / 'combined_bias.fit', overwrite=True)

INFO: splitting each image into 11 chunks to limit memory usage to 350000000.0 bytes. [ccdproc.combiner]


### Dark Frame Cleaning

In [5]:
path_dark_raw = 'data/darks'
combined_bias = astropy.nddata.CCDData.read('data/bias/combined_bias.fit', unit='adu')

for file_path in glob.glob(os.path.join(path_dark_raw,'*.fits')):
    ccd = ccdp.CCDData.read(file_path, unit='adu')
    ccd = ccdp.subtract_bias(ccd, combined_bias)
    file_name = os.path.basename(file_path)
    ccd.write('data/darks/bias_subtracted_' + file_name, overwrite = True)

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


In [13]:
path_subtracted_dark_data = Path('data/darks')

dark_file_paths = glob.glob(str(path_subtracted_dark_data / 'bias_subtracted_*.dark.*.fits'))

combined_darks = ccdp.combine(dark_file_paths,
                             method='median',
                             mem_limit=350e6,
                             unit='adu'
                            )

combined_darks.meta['combined'] = True
combined_darks.write('data/darks/combined_darks.fit', overwrite=True)

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: splitting each image into 23 chunks to limit memory usage to 350000000.0 bytes. [ccdproc.combiner]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the uni

### Cleaning B-Filter Flats

In [ ]:
def normalise(arr, mean):
    arr.data[:] = arr.data / mean
    return arr

combined_bias = ccdp.CCDData.read('data/bias/combined_bias.fit', unit='adu')
combined_darks = ccdp.CCDData.read('data/darks/combined_darks.fit', unit='adu')

seconds_unit = u.s

for file_path in glob.glob(os.path.join('data/flat_B/*.fits')):
    ccd = ccdp.CCDData.read(file_path, unit='adu')
    ccd = ccdp.subtract_bias(ccd, combined_bias)
    ccd = ccdp.subtract_dark(ccd, combined_darks, exposure_time='EXPTIME', exposure_unit = seconds_unit)
    ccd_mean =  np.mean(ccd)
    ccd = normalise(ccd, ccd_mean)
    file_name = os.path.basename(file_path)
    ccd.write('data/flat_B/cleaned_' + file_name, overwrite = True)

In [ ]:
calibrated_flats = glob.glob('data/flat_B/*.fits')

combined_flats = ccdp.combine(calibrated_flats,
                             method='median',
                             mem_limit=350e6,
                             unit = 'adu'
                            )

combined_flats.meta['combined'] = True
combined_flats.write('data/flat_B/combined_flats_B.fits', overwrite = True)

### Cleaning the R Filter

In [4]:
def normalise(arr, mean):
    arr.data[:] = arr.data / mean
    return arr

combined_bias = ccdp.CCDData.read('data/bias/combined_bias.fit', unit='adu')
combined_darks = ccdp.CCDData.read('data/darks/combined_darks.fit', unit='adu')

seconds_unit = u.s

for file_path in glob.glob(os.path.join('data/flat_R/*.fits')):
    ccd = ccdp.CCDData.read(file_path, unit='adu')
    ccd = ccdp.subtract_bias(ccd, combined_bias)
    ccd = ccdp.subtract_dark(ccd, combined_darks, exposure_time='EXPTIME', exposure_unit = seconds_unit)
    ccd_mean =  np.mean(ccd)
    ccd = normalise(ccd, ccd_mean)
    file_name = os.path.basename(file_path)
    ccd.write('data/flat_R/cleaned_' + file_name, overwrite = True)

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


In [7]:
calibrated_flats = glob.glob('data/flat_R/*.fits')

combined_flats = ccdp.combine(calibrated_flats,
                             method='median',
                             mem_limit=350e6,
                             unit = 'adu'
                            )

combined_flats.meta['combined'] = True
combined_flats.write('data/flat_R/combined_flats_R.fits', overwrite = True)

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: splitting each image into 16 chunks to limit memory usage to 350000000.0 bytes. [ccdproc.combiner]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the uni

### Cleaning the R band Science Images

In [9]:
combined_dark = ccdp.CCDData.read('data/darks/combined_darks.fit', unit='adu')
combined_bias = ccdp.CCDData.read('data/bias/combined_bias.fit', unit='adu')
combined_flat = ccdp.CCDData.read('data/flat_R/combined_flats_R.fits', unit='adu')
cleaned_science_path = Path('data/science_R')

i = 36
for file_path in cleaned_science_path.glob('*.fits'):
    ccd = ccdp.CCDData.read(file_path, unit='adu')
    ccd = ccdp.subtract_bias(ccd, combined_bias)
    ccd = ccdp.subtract_dark(ccd, combined_dark, exposure_time='EXPTIME', exposure_unit = seconds_unit, scale = True)
    ccd = ccdp.ccd_process(ccd, master_flat = combined_flat, exposure_key='EXPTIME', exposure_unit = seconds_unit)
    ccd.write(cleaned_science_path / f'{i}.fits', overwrite = True)
    i += 1

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


### Cleaning the B band Science Images

In [13]:
combined_dark = ccdp.CCDData.read('data/darks/combined_darks.fit', unit='adu')
combined_bias = ccdp.CCDData.read('data/bias/combined_bias.fit', unit='adu')
combined_flat = ccdp.CCDData.read('data/flat_B/combined_flats_B.fit', unit='adu')
cleaned_science_path = Path('data/science_B')

i = 31
for file_path in cleaned_science_path.glob('*.fits'):
    ccd = ccdp.CCDData.read(file_path, unit='adu')
    ccd = ccdp.subtract_bias(ccd, combined_bias)
    ccd = ccdp.subtract_dark(ccd, combined_dark, exposure_time='EXPTIME', exposure_unit = seconds_unit, scale = True)
    ccd = ccdp.ccd_process(ccd, master_flat = combined_flat, exposure_key='EXPTIME', exposure_unit = seconds_unit)
    ccd.write(cleaned_science_path / f'{i}.fits', overwrite = True)
    i += 1

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
